## Question 

Develop a basic movie recommendation system that scrapes data from the IMDb website. The program will retrieve information from the Most Popular Movies page on IMDb. The data to be scraped is

1) Movie Titles <br>
2) Runtime <br>
3) Rating <br>
4) Age Restriction <br>
5) Genre <br>
6) Writer(s) <br>
7) Director(s) <br>
8) Fun Movie Trivia <br> 

The data will be stored in a Pandas dataframe.

Subsequently, the program will prompt the user to specify their preferences for:

1) Genre <br>
2) Minimum Rating<br>
3) Maximum Runtime<br>
4) Age Restriction<br>

It will then filter the dataframe accordingly and select a movie for recommendation. The recommendation will display the movie title, writer, director, and trivia. If there are no movies that match the user's preferences, the program will recommend any movie from the original dataframe.

In [62]:
import time
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.common.exceptions import StaleElementReferenceException
from selenium.webdriver.common.action_chains import ActionChains 

movie=[]
runtimes=[]
ratings=[]
ages=[]
genres=[]
writers=[]
directors=[]
trivias=[]
year=[]
rating=[]

driver = webdriver.Chrome()
driver.get('https://www.imdb.com/chart/moviemeter/')


mainBox = driver.find_element(By.XPATH, '/html/body/div[2]/main/div/div[3]/section/div/div[2]/div/ul')

# names is holding the name/title of every movie link to open every movie

names = mainBox.find_elements(By.CLASS_NAME, 'ipc-title__text')
data=len(names)
naam=[]
for i in names:
    # print(i.text)
    naam.append(i.text)

# for i in range(data):
for i in range(25):
    

    driver.execute_script(f"arguments[0].scrollIntoView();", names[i])

    # driver.execute_script(f"window.scrollTo(0, {i * 300});")
    
    # Click on the movie link
    names[i].click()

    time.sleep(0.5)
    try:

    # Extract information for the current movie
        upperBox = driver.find_element(By.XPATH, '//*[@id="__next"]/main/div/section[1]/section/div[3]/section/section/div[2]')

        YearAgeTime = upperBox.find_element(By.XPATH, '//*[@id="__next"]/main/div/section[1]/section/div[3]/section/section/div[2]/div[1]/ul')

        info_list = YearAgeTime.text.split('\n')

        # Check if there are enough elements in the list
        if len(info_list) >= 2:
            year.append(YearAgeTime.text.split('\n')[0])
            ages.append(YearAgeTime.text.split('\n')[1])
            # You can handle other elements similarly
            # runtimes.append(YearAgeTime.text.split('\n')[2])
            runtime = YearAgeTime.text.split('\n')
            runtimes = runtime[2] if len(runtime) > 2 else None
        else:
            print(f"Insufficient data for movie {i + 1}. Skipping...")


        # print(i)
        writers_block = driver.find_element(By.XPATH, '//*[@id="__next"]/main/div/section[1]/section/div[3]/section/section/div[3]/div[2]/div[1]/section/div[2]/div/ul/li[2]')
        writersBlock = writers_block.find_elements(By.TAG_NAME, 'a')
        
        for writer in writersBlock:
            writers.append(writer.text)


        director = driver.find_element(By.XPATH,'//*[@id="__next"]/main/div/section[1]/section/div[3]/section/section/div[3]/div[2]/div[1]/section/div[2]/div/ul/li[1]/div/ul/li/a')
        director = director.text
        directors.append(director)
    
        box_genre = driver.find_element(By.XPATH, '//*[@id="__next"]/main/div/section[1]/section/div[3]/section/section/div[3]/div[2]/div[1]/section/div[1]/div[2]')
        genre_lst = box_genre.text.split('\n')
        genres.append(genre_lst)

    except StaleElementReferenceException:
        print(f"StaleElementReferenceException occurred. Skipping movie {i + 1}")


    driver.back()
    time.sleep(0.5)  # You might need to adjust the sleep duration

    mainBox = driver.find_element(By.XPATH, '/html/body/div[2]/main/div/div[3]/section/div/div[2]/div/ul')
    names = mainBox.find_elements(By.CLASS_NAME, 'ipc-title__text')
    
    
    




Insufficient data for movie 5. Skipping...
Insufficient data for movie 16. Skipping...


In [67]:
import pandas as pd

min_length = min(len(naam), len(genres), len(year), len(ages), len(writers))

# Create a dictionary with the same length for all lists
data = {
    'Naam': naam[:min_length],
    'Genres': genres[:min_length],
    'Year': year[:min_length],
    'Ages': ages[:min_length],
    'Director':directors[:min_length],
    'Writers': writers[:min_length]#,
    # 'Rating':rating[:min_length]

}

df = pd.DataFrame(data)

# Display the DataFrame
df


,Naam,Genres,Year,Ages,Director,Writers
0,The Killer,"[Action, Adventure, Crime]",2023,15,David Fincher,Alexis Nolent
1,The Marvels,"[Action, Adventure, Fantasy]",2023,12A,Nia DaCosta,Luc Jacamon
2,The Hunger Games: The Ballad of Songbirds & Sn...,"[Action, Adventure, Drama]",2023,12A,Francis Lawrence,Andrew Kevin Walker
3,Tiger 3,"[Action, Adventure, Thriller]",2023,2h 34m,Maneesh Sharma,Nia DaCosta
4,Madame Web,"[Action, Adventure, Sci-Fi]",2023,15,S.J. Clarkson,Megan McDonnell
5,Five Nights at Freddy's,"[Horror, Mystery, Thriller]",2023,15,Emma Tammi,Elissa Karasik
6,Oppenheimer,"[Biography, Drama, History]",2023,12A,Christopher Nolan,Michael Lesslie
7,The Creator,"[Action, Adventure, Drama]",2023,15,Gareth Edwards,Michael Arndt
8,Killers of the Flower Moon,"[Crime, Drama, History]",2024,1h 25m,Martin Scorsese,Suzanne Collins
9,Damsel,"[Action, Adventure, Fantasy]",2023,18,Juan Carlos Fresnadillo,Shridhar Raghavan


In [140]:
movie=[]
runtimes=[]
ratings=[]
ages=[]
genres=[]
writers=[]
directors=[]
trivias=[]

mainBox=driver.find_element(By.XPATH, '/html/body/div[2]/main/div/div[3]/section/div/div[2]/div/ul')


# names is holding the name/title of every movie link to open every movie 
names=mainBox.find_elements(By.CLASS_NAME,'ipc-title__text')
print(names[1].text)
names[0].click()



The Marvels


In [141]:
upperBox=driver.find_element(By.XPATH,'//*[@id="__next"]/main/div/section[1]/section/div[3]/section/section/div[2]')


year=[]
rating=[]
time=[]

YearAgeTime=upperBox.find_element(By.XPATH,'//*[@id="__next"]/main/div/section[1]/section/div[3]/section/section/div[2]/div[1]/ul')
YearAgeTime.text

'2023\n15\n1h 58m'

In [108]:
YearAgeTime.text.split('\n')[0]

'2023'

In [109]:
YearAgeTime.text.split('\n')[1]

'15'

In [110]:
YearAgeTime.text.split('\n')[2]

# year, age, and time is done 
#now make them in a loop for all movies

'1h 58m'

In [112]:
rector=driver.find_element(By.CLASS_NAME,'ipc-metadata-list-item__content-container')
rector.text

'David Fincher'

In [113]:
writers_block = driver.find_element(By.XPATH,'//*[@id="__next"]/main/div/section[1]/section/div[3]/section/section/div[3]/div[2]/div[1]/section/div[2]/div/ul/li[2]')
writers_block.text.split('\n')[1]
writersBlock = writers_block.find_elements(By.TAG_NAME, 'a')
for writer in writersBlock:
    writers.append(writer.text)

writers

['Alexis Nolent', 'Luc Jacamon', 'Andrew Kevin Walker']

In [114]:
box_genre = driver.find_element(By.XPATH,'//*[@id="__next"]/main/div/section[1]/section/div[3]/section/section/div[3]/div[2]/div[1]/section/div[1]/div[2]')
genre_lst = box_genre.text.split('\n')
genre_lst

['Action', 'Adventure', 'Crime']

In [ ]:
r1 = "This abnormal is a bad product, i don't like it at all"
r2 = 'the finishing was very bad and product looked used'
r3 = 'i hate the cover which came with it'

# Create a list to store reviews
all_reviews = [r1, r2, r3]

In [ ]:
sentiment_info = []
import string

# Clean the reviews and analyze sentiment
for review in all_reviews:
    # Tokenize the review into words
    tokenized_review = nltk.word_tokenize(review)

    # Convert each word to lowercase
    lowercase_words = [word.lower() for word in tokenized_review]

    # Remove stopwords and punctuation
    cleaned_review = [word for word in lowercase_words if word not in stopwords.words('english') and word not in string.punctuation]

    # Count the number of positive and negative words in the cleaned review
    pos_count = 0
    neg_count = 0

    for word in cleaned_review:
        if word in all_pos_words:
            pos_count += 1
        elif word in all_neg_words:
            neg_count += 1

    # Determine the sentiment label based on the counts
    if pos_count > neg_count:
        sentiment_label = 'positive'
    elif pos_count < neg_count:
        sentiment_label = 'negative'
    else:
        sentiment_label = 'neutral'

    sentiment_info.append({
        'Sentiment Label': sentiment_label,
        'Positive Word Count': pos_count,
        'Negative Word Count': neg_count
    })

# Display the sentiment labels and counts
for info in sentiment_info:
    print(info)